In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
from tensorflow.keras.models import load_model
import pickle
model = load_model('asl_cnn_model.h5')
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

label_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
              'n', 'nothing', 'o', 'p', 'q', 'r', 's', 'space', 't', 'u', 'v', 
              'w', 'x', 'y', 'z']

print("Label list loaded:", label_list)


cap = cv2.VideoCapture(0)

def detect_sign():
    with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.8) as hands:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break  
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb = cv2.flip(frame_rgb, 1)  
            frame_rgb.flags.writeable = False
            results = hands.process(frame_rgb)
            frame_rgb.flags.writeable = True
            frame_rgb = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(frame_rgb, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    data_aux = []
                    for landmark in hand_landmarks.landmark:
                        data_aux.append(landmark.x)
                        data_aux.append(landmark.y)

                    if len(data_aux) == 42:  
                        input_data = np.array(data_aux).reshape(1, 42, 1)
                        prediction = model.predict(input_data)
                        predicted_label_index = np.argmax(prediction, axis=1)[0]
                        if predicted_label_index < len(label_list):
                            sign_name = label_list[predicted_label_index]
                        else:
                            sign_name = "Unknown Sign"  
                        cv2.putText(frame_rgb, sign_name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.imshow('ASL Detection', frame_rgb)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    detect_sign()
